# Chuẩn Hóa và Làm Sạch Dữ Liệu TFT Leaderboard

1. Kiểm tra kiểu dữ liệu
2. Xử lý giá trị thiếu (missing values)
3. Chuẩn hóa kiểu dữ liệu
4. Lưu dữ liệu đã làm sạch

In [12]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Load dữ liệu CSV
df = pd.read_csv('data/leaderboard.csv')

print(f"Số dòng: {len(df)}")
print(f"Số cột: {len(df.columns)}")
print(f"Kích thước: {df.shape}")
df.head()

Số dòng: 1000
Số cột: 41
Kích thước: (1000, 41)


,rank,rating_numeric,num_played,player_id,summoner_region,riot_id,puuid,rating,live.encryption_key,live.game_start_time,...,stats.currentPatchResult.damage_percentile_sum,stats.currentPatchResult.lpChange,stats.currentPatchResult.num_played,stats.currentPatchResult.percentile_count,stats.currentPatchResult.place_sum,stats.currentPatchResult.topCarries,stats.currentPatchResult.wins,stats.num_played,stats.place_sum,stats.wins
0,1,4872,561,353582905,me1,منجة#بحبك,SBM-65J8LDsup6NqXNrraEppoDoHt_QkPlA2qicbM_wQhT...,CHALLENGER I 2072 LP,NaN,NaN,...,1339.0,280.0,22.0,22.0,55.0,[],9.0,561,1983,109
1,2,4871,477,3941714,br1,KBM Toddy#tft,nx9yc2rDqX_Rc7xwa7hJXeLZSQ5vCK6xMo9E8jDA9KQN1G...,CHALLENGER I 2071 LP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,477,1576,120
2,3,4737,507,1696341,eun1,Litiq#EUNE,h-oGE9-A_UbLbAn4r2Mhw0RST6V2otlLfuMihFFRXsyEOa...,CHALLENGER I 1937 LP,NaN,NaN,...,3.0,10.0,1.0,1.0,4.0,"[{""character_id"": ""TFT15_Samira"", ""count"": 1, ...",0.0,506,1732,119
3,4,4735,477,450383412,sg2,FuuTime#xdd,dfFZTGk0bBYppTBlPNMvwrfsLwpsJF9vgl53-jjWxG4Kjm...,CHALLENGER I 1935 LP,NaN,NaN,...,290.0,83.0,8.0,8.0,22.0,[],2.0,477,1711,121
4,5,4670,688,3322631,la2,BlackSheep#Rap,NiJyBETk-4MDyXRA7qWaK-mPTVeAIdEOQkHr8nH7ywb__m...,CHALLENGER I 1870 LP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,687,2206,211


In [14]:
# Kiểm tra kiểu dữ liệu
print("=== KIỂU DỮ LIỆU CÁC CỘT ===")
print(df.dtypes)
print("\n" + "="*60)

# Thông tin tổng quan
print("\n=== THÔNG TIN TỔNG QUAN ===")
df.info()

=== KIỂU DỮ LIỆU CÁC CỘT ===
rank                                                        int64
rating_numeric                                              int64
num_played                                                  int64
player_id                                                   int64
summoner_region                                            object
riot_id                                                    object
puuid                                                      object
rating                                                     object
live.encryption_key                                        object
live.game_start_time                                      float64
live.match_id                                              object
live.player_id                                            float64
live.type                                                  object
stats.RecentResult.ItemData.AD                              int64
stats.RecentResult.ItemData.AP                 

In [15]:
# Kiểm tra số lượng giá trị thiếu
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Tạo DataFrame để hiển thị
missing_df = pd.DataFrame({
    'Cột': missing_values.index,
    'Số lượng thiếu': missing_values.values,
    'Tỷ lệ (%)': missing_percentage.values
})

# Chỉ hiển thị các cột có giá trị thiếu
missing_df = missing_df[missing_df['Số lượng thiếu'] > 0].sort_values('Số lượng thiếu', ascending=False)

print("=== CÁC CỘT CÓ GIÁ TRỊ THIẾU ===")
if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("Không có giá trị thiếu!")
    
print(f"\n\nTổng số cột có giá trị thiếu: {len(missing_df)}/{len(df.columns)}")

=== CÁC CỘT CÓ GIÁ TRỊ THIẾU ===
                                                   Cột  Số lượng thiếu  Tỷ lệ (%)
                                   live.encryption_key             955       95.5
                                  live.game_start_time             955       95.5
                                         live.match_id             955       95.5
                                        live.player_id             955       95.5
                                             live.type             955       95.5
                  stats.currentPatchResult.ItemData.AD             624       62.4
                stats.currentPatchResult.ItemData.Tank             624       62.4
                  stats.currentPatchResult.ItemData.AP             624       62.4
               stats.currentPatchResult.avg_similarity             624       62.4
stats.currentPatchResult.board_strength_percentile_sum             624       62.4
             stats.currentPatchResult.percentile_count           

In [16]:
# Tạo bản sao để làm việc
df_clean = df.copy()

# Đảm bảo các cột số nguyên
integer_columns = ['rank', 'rating_numeric', 'num_played', 'player_id']
for col in integer_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0).astype('int64')

# Chuẩn hóa các cột chuỗi
string_columns = ['summoner_region', 'riot_id', 'puuid', 'rating']
for col in string_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str)

# Chuẩn hóa các cột số thực cho stats
float_columns = [col for col in df_clean.columns if any(x in col for x in ['avg_similarity', 'percentile', 'lpChange'])]
for col in float_columns:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Chuẩn hóa các cột số nguyên cho stats
stats_int_columns = [col for col in df_clean.columns if any(x in col for x in ['ItemData', 'num_played', 'place_sum', 'wins', 'percentile_count'])]
for col in stats_int_columns:
    if col not in integer_columns:  # Tránh xử lý lại
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0).astype('int64')

print("✓ Đã chuẩn hóa kiểu dữ liệu")
print(f"\nKiểu dữ liệu sau khi chuẩn hóa:")
print(df_clean.dtypes)

✓ Đã chuẩn hóa kiểu dữ liệu

Kiểu dữ liệu sau khi chuẩn hóa:
rank                                                        int64
rating_numeric                                              int64
num_played                                                  int64
player_id                                                   int64
summoner_region                                            object
riot_id                                                    object
puuid                                                      object
rating                                                     object
live.encryption_key                                        object
live.game_start_time                                      float64
live.match_id                                              object
live.player_id                                            float64
live.type                                                  object
stats.RecentResult.ItemData.AD                              int64
stats.RecentRes

In [17]:
# Xử lý giá trị thiếu

# 1. Điền 0 cho các cột số nguyên có giá trị thiếu
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(0)

# 2. Điền chuỗi rỗng cho các cột string có giá trị thiếu  
string_cols = df_clean.select_dtypes(include=['object']).columns
for col in string_cols:
    if df_clean[col].isnull().sum() > 0:
        # Nếu là cột topCarries, điền []
        if 'topCarries' in col:
            df_clean[col] = df_clean[col].fillna('[]')
        else:
            df_clean[col] = df_clean[col].fillna('')

# 3. Xử lý đặc biệt cho cột 'live' (có thể có nhiều missing vì người chơi không online)
live_columns = [col for col in df_clean.columns if col.startswith('live.')]
for col in live_columns:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna('')

print("✓ Đã xử lý giá trị thiếu")
print(f"\nKiểm tra lại giá trị thiếu:")
print(f"Tổng số giá trị thiếu: {df_clean.isnull().sum().sum()}")

if df_clean.isnull().sum().sum() > 0:
    print("\nCác cột vẫn còn giá trị thiếu:")
    print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])
else:
    print("✓ Không còn giá trị thiếu!")

✓ Đã xử lý giá trị thiếu

Kiểm tra lại giá trị thiếu:
Tổng số giá trị thiếu: 0
✓ Không còn giá trị thiếu!


In [18]:
# Tóm tắt dữ liệu sau khi làm sạch
print("=== THỐNG KÊ DỮ LIỆU SAU KHI LÀM SẠCH ===\n")
print(f"Số dòng: {len(df_clean)}")
print(f"Số cột: {len(df_clean.columns)}")
print(f"Tổng giá trị thiếu: {df_clean.isnull().sum().sum()}")

print("\n=== THỐNG KÊ MÔ TẢ CÁC CỘT CHÍNH ===")
main_cols = ['rank', 'rating_numeric', 'num_played', 'stats.num_played', 'stats.wins']
existing_main_cols = [col for col in main_cols if col in df_clean.columns]
print(df_clean[existing_main_cols].describe())

print("\n=== MẪU DỮ LIỆU ===")
df_clean.head()

=== THỐNG KÊ DỮ LIỆU SAU KHI LÀM SẠCH ===

Số dòng: 1000
Số cột: 41
Tổng giá trị thiếu: 0

=== THỐNG KÊ MÔ TẢ CÁC CỘT CHÍNH ===
              rank  rating_numeric   num_played  stats.num_played  stats.wins
count  1000.000000     1000.000000  1000.000000       1000.000000   1000.0000
mean    500.500000     4051.887000   607.092000        607.022000     98.9020
std     288.819436      150.670009   272.230772        272.196834     42.8847
min       1.000000     3897.000000   129.000000        129.000000     28.0000
25%     250.750000     3944.750000   404.750000        404.750000     67.7500
50%     500.500000     4005.000000   572.000000        572.000000     92.0000
75%     750.250000     4109.000000   748.000000        748.000000    120.0000
max    1000.000000     4872.000000  1988.000000       1988.000000    351.0000

=== MẪU DỮ LIỆU ===


,rank,rating_numeric,num_played,player_id,summoner_region,riot_id,puuid,rating,live.encryption_key,live.game_start_time,...,stats.currentPatchResult.damage_percentile_sum,stats.currentPatchResult.lpChange,stats.currentPatchResult.num_played,stats.currentPatchResult.percentile_count,stats.currentPatchResult.place_sum,stats.currentPatchResult.topCarries,stats.currentPatchResult.wins,stats.num_played,stats.place_sum,stats.wins
0,1,4872,561,353582905,me1,منجة#بحبك,SBM-65J8LDsup6NqXNrraEppoDoHt_QkPlA2qicbM_wQhT...,CHALLENGER I 2072 LP,,0.0,...,1339.0,280.0,22,22,55,[],9,561,1983,109
1,2,4871,477,3941714,br1,KBM Toddy#tft,nx9yc2rDqX_Rc7xwa7hJXeLZSQ5vCK6xMo9E8jDA9KQN1G...,CHALLENGER I 2071 LP,,0.0,...,0.0,0.0,0,0,0,[],0,477,1576,120
2,3,4737,507,1696341,eun1,Litiq#EUNE,h-oGE9-A_UbLbAn4r2Mhw0RST6V2otlLfuMihFFRXsyEOa...,CHALLENGER I 1937 LP,,0.0,...,3.0,10.0,1,1,4,"[{""character_id"": ""TFT15_Samira"", ""count"": 1, ...",0,506,1732,119
3,4,4735,477,450383412,sg2,FuuTime#xdd,dfFZTGk0bBYppTBlPNMvwrfsLwpsJF9vgl53-jjWxG4Kjm...,CHALLENGER I 1935 LP,,0.0,...,290.0,83.0,8,8,22,[],2,477,1711,121
4,5,4670,688,3322631,la2,BlackSheep#Rap,NiJyBETk-4MDyXRA7qWaK-mPTVeAIdEOQkHr8nH7ywb__m...,CHALLENGER I 1870 LP,,0.0,...,0.0,0.0,0,0,0,[],0,687,2206,211


In [19]:
# Lưu dữ liệu đã làm sạch
output_file = 'data/leaderboard_cleaned.csv'
df_clean.to_csv(output_file, index=False)

print(f"✓ Đã lưu dữ liệu đã làm sạch vào: {output_file}")
print(f"\nThông tin file:")
print(f"  - Số dòng: {len(df_clean)}")
print(f"  - Số cột: {len(df_clean.columns)}")

✓ Đã lưu dữ liệu đã làm sạch vào: data/leaderboard_cleaned.csv

Thông tin file:
  - Số dòng: 1000
  - Số cột: 41
